<h1 style="color:#65AE11;">复制和计算重叠的注意事项</h1>

在本节中，你将学习执行复制/计算重叠时的数据分块注意事项。

<h2 style="color:#65AE11;">目标</h2>

完成本节内容的学习后，你将：

* 了解如何使用数据分块来实现数据复制与计算的重叠 
* 掌握数据索引技术，该技术使代码具有足够的灵活性，能够处理任意大小的数据和任意数量的流

<h2 style="color:#65AE11;">演示</h2>

运行以下单元以加载本节的视频演示。看完演示后，继续完成此 Notebook 的余下部分。

In [ ]:
%%html
<video width="800" controls>
  <source src="https://dli-lms.s3.us-east-1.amazonaws.com/assets/s-ac-01-v1-zh/08-cc-considerations-01.mp4" type="video/mp4">
  Your browser does not support HTML video.
</video>

<h2 style="color:#65AE11;">复制和计算重叠代码示例</h2>

下方是上述方法的两个代码示例，第一个示例适用于条目数被流数量整除的情况，第二个示例则是不能整除的情况。

<h3 style="color:#65AE11;">N 被流数量整除</h3>

```c
// "Simple" version where number of entries is evenly divisible by number of streams.

// Set to a ridiculously low value to clarify mechanisms of the technique.
const uint64_t num_entries = 10;
const uint64_t num_iters = 1UL << 10;

// Allocate memory for all data entries. Make sure to pin host memory.
cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
cudaMalloc    (&data_gpu, sizeof(uint64_t)*num_entries);

// Set the number of streams.
const uint64_t num_streams = 2;

// Create an array of streams containing number of streams
cudaStream_t streams[num_streams];
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamCreate(&streams[stream]);

// Set number of entries for each "chunk". Assumes `num_entries % num_streams == 0`.
const uint64_t chunk_size = num_entries / num_streams;

// For each stream, calculate indices for its chunk of full dataset and then, HtoD copy, compute, DtoH copy.
for (uint64_t stream = 0; stream < num_streams; stream++) {

    // Get start index in full dataset for this stream's work.
    const uint64_t lower = chunk_size*stream;
    
    // Stream-indexed (`data+lower`) and chunk-sized HtoD copy in the non-default stream
    // `streams[stream]`.
    cudaMemcpyAsync(data_gpu+lower, data_cpu+lower, 
           sizeof(uint64_t)*chunk_size, cudaMemcpyHostToDevice, 
           streams[stream]);
    
    // Stream-indexed (`data_gpu+lower`) and chunk-sized compute in the non-default stream
    // `streams[stream]`.
    decrypt_gpu<<<80*32, 64, 0, streams[stream]>>>
        (data_gpu+lower, chunk_size, num_iters);
    
    // Stream-indexed (`data+lower`) and chunk-sized DtoH copy in the non-default stream
    // `streams[stream]`.
    cudaMemcpyAsync(data_cpu+lower, data_gpu+lower, 
           sizeof(uint64_t)*chunk_size, cudaMemcpyDeviceToHost, 
           streams[stream]);
}

// Destroy streams.
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamDestroy(streams[stream]);
```

<h3 style="color:#65AE11;">N 未被流数量整除</h3>

```c
// Able to handle when `num_entries % num_streams != 0`.

const uint64_t num_entries = 10;
const uint64_t num_iters = 1UL << 10;

cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
cudaMalloc    (&data_gpu, sizeof(uint64_t)*num_entries);

// Set the number of streams to not evenly divide num_entries.
const uint64_t num_streams = 3;

cudaStream_t streams[num_streams];
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamCreate(&streams[stream]);

// Use round-up division (`sdiv`, defined in helper.cu) so `num_streams*chunk_size`
// is never less than `num_entries`.
// This can result in `num_streams*chunk_size` being greater than `num_entries`, meaning
// we will need to guard against out-of-range errors in the final "tail" stream (see below).
const uint64_t chunk_size = sdiv(num_entries, num_streams);

for (uint64_t stream = 0; stream < num_streams; stream++) {

    const uint64_t lower = chunk_size*stream;
    // For tail stream `lower+chunk_size` could be out of range, so here we guard against that.
    const uint64_t upper = min(lower+chunk_size, num_entries);
    // Since the tail stream width may not be `chunk_size`,
    // we need to calculate a separate `width` value.
    const uint64_t width = upper-lower;

    // Use `width` instead of `chunk_size`.
    cudaMemcpyAsync(data_gpu+lower, data_cpu+lower, 
           sizeof(uint64_t)*width, cudaMemcpyHostToDevice, 
           streams[stream]);

    // Use `width` instead of `chunk_size`.
    decrypt_gpu<<<80*32, 64, 0, streams[stream]>>>
        (data_gpu+lower, width, num_iters);

    // Use `width` instead of `chunk_size`.
    cudaMemcpyAsync(data_cpu+lower, data_gpu+lower, 
           sizeof(uint64_t)*width, cudaMemcpyDeviceToHost, 
           streams[stream]);
}

// Destroy streams.
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamDestroy(streams[stream]);
```

<h2 style="color:#65AE11;">知识检查</h2>

请回答下列问题，确认你已了解本节的主要目标。你可以通过单击问题下方的“...”单元来显示每个问题的答案。

---

**若将 3 个不同的非默认流分别用于所有的主机到设备的内存传输、所有的 GPU 计算和所有的设备到主机的内存传输，会有什么问题？**

**答案：**

在不同的非默认流中，操作之间没有必然的顺序。在其各自的非默认流中发布主机到设备的传输、GPU 计算和设备到主机的传输可能会违反以下限制条件：GPU 计算需待主机到设备的传输操作完成后才可执行，且设备到主机的传输需待 GPU 计算完成后才可执行。

---

**执行数据分块在实现复制和计算重叠的过程中为何是一个有效环节？**

**答案：**

对数据进行分块时，我们可以：

1. 通过在同一非默认流中执行主机到设备的数据传输、GPU 计算和设备到主机的数据传输这 3 项操作，可以保证这些操作的正确操作顺序

2. 对不同的数据块使用不同的非默认流，以便能将某个非默认流中的数据复制与另一个非默认流中的计算（以及反向复制）进行重叠。

---

<h2 style="color:#65AE11;">下一步</h2>

现在你已了解如何为任意数量的数据和多个流实现复制和计算重叠，下一节中，你将运用自己的理解，在密码应用中实现复制和计算重叠。

请继续学习下一节：[*练习：应用复制/计算重叠*](../09_Exercise_Apply_Streams/Exercise_Apply_Streams.ipynb)。

<h2 style="color:#65AE11;">选择进一步学习</h2>

以下内容适用于有时间且有兴趣就本研讨会相关主题进行额外研究的学员。

* GPU 技术大会讲稿 [*CUDA 流：最佳实践和常见陷阱*](https://on-demand.gputechconf.com/gtc/2014/presentations/S4158-cuda-streams-best-practices-common-pitfalls.pdf) 将带你回顾本次研讨会中涉及的许多主题和一些其他话题，以及使用流时可能出错的常见场景示例。